# N-gram document representations of headlines
This notebook guides you through creating n-gram representations of headlines. We'll examine these representations and use them as features for a simple machine learning model to classify headlines as clickbait or not.

## Import necessary packages
These should be automatically available in the class conda environment. If this doesn't work, make sure you specified `/ix/cs2731_fall2025f/class_env` when launching JupyterHub.

In [ ]:
import sklearn
import pandas
import nltk
nltk.download('punkt_tab')

## Load clickbait data from Kaggle
This data consists of headlines classified as clickbait or not (regular news). It is from a dataset on Kaggle, a site where machine learning competitions and datasets are often hosted. Source site: https://www.kaggle.com/datasets/amananandrai/clickbait-dataset

In [ ]:
# Read in the dataset with pandas
# 0 corresponds to not clickbait, 1 has been judged as clickbait

import pandas as pd

# Set pandas to display entire texts in dataframes
pd.set_option('display.max_colwidth', None)

data = pd.read_csv('data/clickbait_data.csv')
data.info()
data.head()

## Split into training and test sets

In [ ]:
from sklearn.model_selection import train_test_split

test_size = int(0.1 * len(data))
train, test  = train_test_split(data, test_size=test_size, random_state=9)
print(len(train))
print(len(test))

**What does this "length" refer to?**

## Extract **n-gram features** from the raw text data
Features are data fields or attributes extracted from raw data, in our case, text data. The features were are examining here are "unigram" features, counts of the number of instances of each word type. This step converts each headline to a numeric vector of unigram counts (how many times each word type occurs).
"Training" the vectorizer means finding how many unique features (in this case, word types) are in the training set. This sets the number of columns in the term-document matrix.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk

unigram_vectorizer = CountVectorizer(tokenizer=nltk.word_tokenize)
unigram_vectorizer.fit(train['headline']) # input is a list of strings (documents)
train_unigram_features = unigram_vectorizer.transform(train['headline'])
test_unigram_features = unigram_vectorizer.transform(test['headline'])

print(type(train_unigram_features))
print(train_unigram_features.shape) # prints (number of rows in the matrix, number of columns)
print(test_unigram_features.shape)  # prints (number of rows in the matrix, number of columns)

Let's explore those training set unigram features a bit more. First convert the `scipy` sparse matrix into a regular `numpy` matrix to take a look at it.

In [ ]:
unigram_features = train_unigram_features.toarray()
print(type(unigram_features))
print(unigram_features.shape)

Each row in this matrix is a headline. Each column is the count of a unique word type. **How many words are there in the entire vocabulary of this dataset?**  
Let's take a look at a few example headline vectors.

In [ ]:
sample_index =  # FILL IN a random number less than the number of rows (datapoints) in ngram_features here
train.iloc[sample_index] # Take a look at the text

How many values in this large, sparse vector that represents this sample document aren't 0?

In [ ]:
import numpy as np

np.count_nonzero(unigram_features[sample_index])

Label the nonzero features with the words they correspond to:

In [ ]:
# Make a pandas dataframe from the ngram features and label the column with their corresponding feature (unigram or word type)
feature_names = unigram_vectorizer.get_feature_names_out()
print(len(feature_names))

unigram_feature_matrix = pd.DataFrame(unigram_features, columns=feature_names)
unigram_feature_matrix

In [ ]:
# View the nonzero values in the feature vector for the example headline
column_mask = unigram_feature_matrix.loc[sample_index].apply(lambda x: x > 0)
nonzero_columns = column_mask[column_mask == True]
unigram_feature_matrix.loc[[sample_index], nonzero_columns.index]

## Extract bigram features
Unigram features are counts of word types, which can also be viewed as unique sequences of just 1 word. Here we use **bigram** features, which are counts of unique sequences of 2 words.

In [ ]:
bigram_vectorizer = CountVectorizer(ngram_range=(2,2), tokenizer=nltk.word_tokenize) # note the ngram_range parameter
bigram_vectorizer.fit(train['headline'])
train_bigram_features = bigram_vectorizer.transform(train['headline'])
test_bigram_features = bigram_vectorizer.transform(test['headline'])

print(train_bigram_features.shape) # prints (number of rows in the matrix, number of columns)

In [ ]:
bigram_feature_matrix = pd.DataFrame(train_bigram_features.toarray(), columns=bigram_vectorizer.get_feature_names_out())
bigram_feature_matrix

**How many bigram features do we have? Why is this so much bigger than the number of unigram features?**

In [ ]:
# View the nonzero values in the feature vector for the example headline
column_mask = bigram_feature_matrix.loc[sample_index].apply(lambda x: x > 0)
nonzero_columns = column_mask[column_mask == True]
bigram_feature_matrix.loc[[sample_index], nonzero_columns.index]

# Cosine similarity
We can use the numeric feature vectors computed for every headline to compute similarities with other headlines using **cosine similarity**. Though contemporary information retreival (search engine) systems are of course much more complex, they still use this basic framework to return results: convert texts to vectors and return the most similar documents to your query.

In [ ]:
# Compute cosine similarity between the sample headline vector and all other headlines in the training set

from scipy.spatial.distance import cosine # cosine distance from the scipy package

In [ ]:
sample_vector = unigram_features[sample_index]
sample_vector

In [ ]:
def compute_cosine_similarity_to_sample(vector):
    """ Compute cosine similarity with sample vector """
    return 1 - cosine(vector, sample_vector)

As a refresher, let's take a look at the shapes of these matrices and what the sample headline is.

In [ ]:
unigram_feature_matrix.shape

In [ ]:
train.iloc[sample_index] # Take a look at the text

In [ ]:
sample_vector.shape

In [ ]:
similarities = unigram_feature_matrix.apply(compute_cosine_similarity_to_sample, axis=1) # apply function over every row in the df
similarities

A sanity check first. What should the sample vector's similarity with itself be?

In [ ]:
similarities[sample_index]

Now let's rank similarities and find out which vectors are most similar to the sample headline

In [ ]:
sorted_similarities = similarities.sort_values(ascending=False)
sorted_similarities

In [ ]:
train.iloc[sorted_similarities.index[:10]] # Take a look at the top 10 most similar headlines

# Clickbait classification
Now let's create a simple machine learning model to classify whether headlines are clickbait or not. We'll use the n-gram features we extracted as input!

Do you n-gram features will be enough to classify clickbait? **What information is lost by representing documents as ngrams?**

## Train Naive Bayes model
Naive Bayes is a simple machine learning algorithm that we won't be covering in the course. But you know that as a machine learning model, it learns patterns (parameters in a mathematical model) from a training set.

That trained model can then be used to make predictions.

In our dataset, `train['clickbait']` is the column that contains the **output** we care about: whether the text is clickbait or not. We pass the example input and output in our training set to train the model.

You can select whether to use unigram or bigram features. Try them both!

In [ ]:
from sklearn.naive_bayes import MultinomialNB

clf = MultinomialNB()
train_features =  # FILL IN with the variable name of the training features to use (unigrams or bigrams)
clf.fit(train_features, train['clickbait'])

## Make predictions from the trained model

In [ ]:
# Select an example from the test set
# Note that the classifier has never seen this example

example = test.sample(1)
example

Get the text into a vector format that the classifier can use.

In [ ]:
vectorizer =  # FILL IN the variable name of the vectorizer to use (unigram or bigram)
example_features = unigram_vectorizer.transform(example['headline'])
example_features.shape

In [ ]:
clf.predict(example_features)

**Did the model correctly predict the unseen example?**

## Evaluation on a test set
Now let's quantitatively evaluate our model on the **test set**.

In [ ]:
import numpy as np

test_labels = test['clickbait'] # true (gold) test set labels for clickbait/not clickbait

test_features =  # FILL IN with variable name of test features to use (unigrams or bigrams)
test_predictions = clf.predict(test_features)

np.mean(test_predictions == test_labels) # a quick measure of accuracy